# Dependencies da scaricare

In [ ]:
# An OpenAI Gym environment for Super Mario Bros. & Super Mario Bros. 2
%pip install gym-super-mario-bros==7.3.0 nes_py

In [ ]:
# CUDA VERSION - An open source machine learning framework that accelerates the path from research prototyping to production deployment
%pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# CPU VERSION - An open source machine learning framework that accelerates the path from research prototyping to production deployment
%pip3 install torch==1.10.2+cpu torchvision==0.11.3+cpu torchaudio==0.10.2+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

In [ ]:
# Reinforcement Learning library
%pip install stable-baselines3
%pip install atari-py

# Import dei moduli

In [ ]:
# Import del gioco
import gym_super_mario_bros
# Import dell'interfaccia del controller
from nes_py.wrappers import JoypadSpace
# Import di una versione semplificata dei controlli
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
# Importa l'interfaccia per l'analisi dell'immagine e della scala di grigi
from gym.wrappers import FrameStack, GrayScaleObservation
# Importa l'interfaccia per la vettorizzazione
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Importa la libreria per la visualizzazione dell'impatto del frame stacking
from matplotlib import pyplot as plt
# Import di OS per la gestione dei file
import os
# Import dell'algoritmo PPO
from stable_baselines3 import PPO
# Import di Base Callbacks per il salvataggio di modelli
from stable_baselines3.common.callbacks import BaseCallback

# Preparazione dell'ambiente

In [ ]:
# Configurazione del gioco
env = gym_super_mario_bros.make('SuperMarioBros-v0') # Definisce l'ambiente di gioco
env = JoypadSpace(env, SIMPLE_MOVEMENT) # Limita le azioni effettuabili a quelle necessarie per procedere nel gioco
env = GrayScaleObservation(env, keep_dim=True) # Osserva la scala di grigi
env = DummyVecEnv([lambda: env]) # Posiziona l'agente in un ambiente di prova
env = VecFrameStack(env, 4, channels_order='last') # Determina quanti frame inserire nello stack e come

# Esecuzione del gioco e caricamento del model

In [ ]:
# Carica il modello
model = PPO.load('./train/best_model_100000') # Specificare il nome del file da caricare

# Avvia il gioco
state = env.reset()
while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()

In [6]:
# Termina il gioco
env.close()

# Apprendimento

In [ ]:
# Definizione della procedura per la creazione del model
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

# Directory per il salvataggio dei dati
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

# Procedura di callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

# Definizione dei parametri del modello di apprendimento
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

In [ ]:
# Allena il modello
model.learn(total_timesteps=1000, callback=callback) # Il parametro total_timestamps indica il numero di step totali dell'apprendimento